# Universal Orlando Wait Time Analysis

This notebook explores wait time patterns across Universal Orlando's theme parks:
- Islands of Adventure
- Universal Studios Florida
- Epic Universe

**Goals:**
1. Understand data quality and coverage
2. Identify wait time patterns by hour and day
3. Compare parks and rides
4. Find the best times to visit

## Setup

In [ ]:
import sqlite3
from pathlib import Path

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Path to database
DB_PATH = Path('../data/wait_times.db')

print(f"Database exists: {DB_PATH.exists()}")

## Load Data

In [ ]:
# Connect to database and load all tables
conn = sqlite3.connect(DB_PATH)

# Load parks
parks_df = pd.read_sql_query("SELECT * FROM parks", conn)
print(f"Parks: {len(parks_df)}")
display(parks_df)

# Load rides
rides_df = pd.read_sql_query("""
    SELECT r.*, p.name as park_name, l.name as land_name
    FROM rides r
    JOIN parks p ON r.park_id = p.id
    LEFT JOIN lands l ON r.land_id = l.id
""", conn)
print(f"\nRides: {len(rides_df)}")

# Load wait times with ride and park info
wait_times_df = pd.read_sql_query("""
    SELECT 
        wt.*,
        r.name as ride_name,
        p.name as park_name,
        l.name as land_name
    FROM wait_times wt
    JOIN rides r ON wt.ride_id = r.id
    JOIN parks p ON r.park_id = p.id
    LEFT JOIN lands l ON r.land_id = l.id
""", conn)

conn.close()

# Convert timestamp columns
wait_times_df['collected_at'] = pd.to_datetime(wait_times_df['collected_at'])

print(f"\nWait time records: {len(wait_times_df):,}")

## Data Overview

In [ ]:
# Basic statistics
print("=" * 50)
print("DATA COLLECTION SUMMARY")
print("=" * 50)

if len(wait_times_df) > 0:
    print(f"Date range: {wait_times_df['collected_at'].min()} to {wait_times_df['collected_at'].max()}")
    print(f"Total records: {len(wait_times_df):,}")
    print(f"Unique rides: {wait_times_df['ride_id'].nunique()}")
    print(f"Parks: {wait_times_df['park_name'].nunique()}")
    
    # Records per park
    print("\nRecords by park:")
    print(wait_times_df.groupby('park_name').size().to_string())
else:
    print("No data collected yet. Run the collector first!")
    print("  python -m src.collector")

In [ ]:
# Rides by park
print("\nRides by park:")
for park_name in rides_df['park_name'].unique():
    park_rides = rides_df[rides_df['park_name'] == park_name]
    print(f"\n{park_name} ({len(park_rides)} rides):")
    for _, ride in park_rides.head(10).iterrows():
        land = f" ({ride['land_name']})" if pd.notna(ride['land_name']) else ""
        print(f"  - {ride['name']}{land}")
    if len(park_rides) > 10:
        print(f"  ... and {len(park_rides) - 10} more")

## Wait Time Distribution

In [ ]:
# Filter to only open rides with valid wait times
open_rides = wait_times_df[(wait_times_df['is_open'] == 1) & (wait_times_df['wait_time'].notna())].copy()

if len(open_rides) > 0:
    print(f"Records with valid wait times: {len(open_rides):,} ({len(open_rides)/len(wait_times_df)*100:.1f}%)")
    
    # Overall distribution
    fig = px.histogram(
        open_rides, 
        x='wait_time',
        nbins=50,
        title='Distribution of Wait Times (All Parks)',
        labels={'wait_time': 'Wait Time (minutes)', 'count': 'Frequency'},
        color_discrete_sequence=['#1f77b4']
    )
    fig.update_layout(showlegend=False)
    fig.show()
else:
    print("No valid wait time data yet.")

In [ ]:
# Distribution by park
if len(open_rides) > 0:
    fig = px.box(
        open_rides,
        x='park_name',
        y='wait_time',
        title='Wait Time Distribution by Park',
        labels={'park_name': 'Park', 'wait_time': 'Wait Time (minutes)'},
        color='park_name'
    )
    fig.update_layout(showlegend=False)
    fig.show()

## Wait Times by Hour of Day

In [ ]:
if len(open_rides) > 0:
    # Average wait time by hour
    hourly_avg = open_rides.groupby(['hour', 'park_name'])['wait_time'].mean().reset_index()
    
    fig = px.line(
        hourly_avg,
        x='hour',
        y='wait_time',
        color='park_name',
        title='Average Wait Time by Hour of Day',
        labels={'hour': 'Hour of Day', 'wait_time': 'Average Wait (minutes)', 'park_name': 'Park'},
        markers=True
    )
    fig.update_xaxes(tickmode='linear', tick0=0, dtick=2)
    fig.show()

## Wait Times by Day of Week

In [ ]:
if len(open_rides) > 0:
    # Map day numbers to names
    day_names = {0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday', 
                 4: 'Friday', 5: 'Saturday', 6: 'Sunday'}
    open_rides['day_name'] = open_rides['day_of_week'].map(day_names)
    
    # Average by day of week
    daily_avg = open_rides.groupby(['day_of_week', 'day_name', 'park_name'])['wait_time'].mean().reset_index()
    daily_avg = daily_avg.sort_values('day_of_week')
    
    fig = px.bar(
        daily_avg,
        x='day_name',
        y='wait_time',
        color='park_name',
        barmode='group',
        title='Average Wait Time by Day of Week',
        labels={'day_name': 'Day', 'wait_time': 'Average Wait (minutes)', 'park_name': 'Park'},
        category_orders={'day_name': list(day_names.values())}
    )
    fig.show()

## Heatmap: Hour vs Day of Week

In [ ]:
if len(open_rides) > 0:
    # Create heatmap data
    heatmap_data = open_rides.pivot_table(
        values='wait_time',
        index='day_of_week',
        columns='hour',
        aggfunc='mean'
    )
    
    # Rename index to day names
    heatmap_data.index = [day_names[i] for i in heatmap_data.index]
    
    fig = px.imshow(
        heatmap_data,
        title='Average Wait Time: Hour vs Day of Week (All Parks)',
        labels={'x': 'Hour of Day', 'y': 'Day of Week', 'color': 'Wait (min)'},
        color_continuous_scale='RdYlGn_r',  # Red = busy, Green = quiet
        aspect='auto'
    )
    fig.show()

## Top Rides by Average Wait Time

In [ ]:
if len(open_rides) > 0:
    # Top 15 rides by average wait
    top_rides = open_rides.groupby(['ride_name', 'park_name']).agg({
        'wait_time': ['mean', 'max', 'count']
    }).round(1)
    top_rides.columns = ['avg_wait', 'max_wait', 'data_points']
    top_rides = top_rides.sort_values('avg_wait', ascending=False).head(15).reset_index()
    
    fig = px.bar(
        top_rides,
        y='ride_name',
        x='avg_wait',
        color='park_name',
        orientation='h',
        title='Top 15 Rides by Average Wait Time',
        labels={'ride_name': 'Ride', 'avg_wait': 'Average Wait (minutes)', 'park_name': 'Park'}
    )
    fig.update_layout(yaxis={'categoryorder': 'total ascending'})
    fig.show()
    
    print("\nTop 15 rides summary:")
    display(top_rides)

## Weekend vs Weekday Comparison

In [ ]:
if len(open_rides) > 0:
    open_rides['period'] = open_rides['is_weekend'].map({0: 'Weekday', 1: 'Weekend'})
    
    weekend_comparison = open_rides.groupby(['period', 'park_name'])['wait_time'].mean().reset_index()
    
    fig = px.bar(
        weekend_comparison,
        x='park_name',
        y='wait_time',
        color='period',
        barmode='group',
        title='Weekend vs Weekday Average Wait Times',
        labels={'park_name': 'Park', 'wait_time': 'Average Wait (minutes)', 'period': 'Period'}
    )
    fig.show()

## Key Insights

Based on the data collected so far:

### 1. Best Times to Visit
*To be filled in once we have more data*

### 2. Busiest Rides
*To be filled in once we have more data*

### 3. Weekend vs Weekday Patterns
*To be filled in once we have more data*

## Data Quality Check

In [ ]:
if len(wait_times_df) > 0:
    print("DATA QUALITY SUMMARY")
    print("=" * 50)
    
    # Missing values
    print(f"\nRecords with null wait_time: {wait_times_df['wait_time'].isna().sum():,}")
    print(f"Records where ride closed: {(wait_times_df['is_open'] == 0).sum():,}")
    
    # Collection frequency
    if len(wait_times_df) > 1:
        time_diffs = wait_times_df.groupby('ride_id')['collected_at'].diff().dropna()
        if len(time_diffs) > 0:
            avg_interval = time_diffs.mean()
            print(f"\nAverage collection interval: {avg_interval}")
    
    # Data points per ride
    points_per_ride = wait_times_df.groupby('ride_name').size()
    print(f"\nData points per ride: min={points_per_ride.min()}, max={points_per_ride.max()}, avg={points_per_ride.mean():.1f}")